In [ ]:
# January-kenkyuu GPU/CPU フルパイプライン実行ノート

# このノートの目的:
# - Google Drive をマウント
# - リポジトリを clone して January-kenkyuu ブランチをチェックアウト
# - 依存パッケージをインストール
# - CLI ランナー (scripts/run_full_pipeline_cli.py) を CPU / GPU で実行

from pathlib import Path

print("Notebook for January-kenkyuu full pipeline (CPU/GPU) setup")

In [ ]:
# 1. Google Drive をマウント

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# 2. リポジトリを clone し、January-kenkyuu ブランチをチェックアウト

%cd /content

# TODO: あなたの GitHub リポジトリURL に書き換えてください
REPO_URL = "https://github.com/your-account/kenkyuu.git"

!git clone "$REPO_URL" kenkyuu
%cd /content/kenkyuu

!git fetch
!git checkout January-kenkyuu


In [ ]:
# 3. 依存パッケージのインストール

%pip install -q -r requirements.txt

# GPU 用ライブラリ（必要になったらコメントアウトを外す）
# 例: CuPy CUDA 12 系
# %pip install -q cupy-cuda12x

In [ ]:
# 4. 入力フォルダ / 出力フォルダの設定（Google Drive 上）

from pathlib import Path

# TODO: あなたの Drive 上の TIF フォルダに書き換えてください
INPUT_FOLDER = "/content/drive/MyDrive/ct_data/your_tif_folder"

OUTPUT_BASE = "/content/drive/MyDrive/kenkyuu_results"
Path(OUTPUT_BASE).mkdir(parents=True, exist_ok=True)

print("INPUT_FOLDER =", INPUT_FOLDER)
print("OUTPUT_BASE =", OUTPUT_BASE)

In [ ]:
# 5. まずは CPU モードでフルパイプラインを 1 回だけ実行して確認

!python scripts/run_full_pipeline_cli.py \
  --input "$INPUT_FOLDER" \
  --output "$OUTPUT_BASE/cli_cpu_test" \
  --max-radius 10 \
  --connectivity 6 \
  --threshold-method otsu \
  --tau-ratio 0.05 \
  --tau-gain-rel 0.003 \
  --contacts-min 4 \
  --contacts-max 10


In [ ]:
# 6. （オプション）GPU モードで複数パラメータを一気に試す例

runs = [
    dict(name="gpu_run_otsu_no_clahe",  threshold="otsu",     clahe=False),
    dict(name="gpu_run_otsu_clahe",     threshold="otsu",     clahe=True),
    dict(name="gpu_run_tri_no_clahe",   threshold="triangle", clahe=False),
]

for cfg in runs:
    out_dir = f"{OUTPUT_BASE}/{cfg['name']}"
    print("========================================")
    print("Running:", cfg["name"])
    print("========================================")

    clahe_flag = "--enable-clahe" if cfg["clahe"] else ""

    # ※ GPU backend 実装後に --backend gpu を有効化する想定
    !python scripts/run_full_pipeline_cli.py \
      --input "$INPUT_FOLDER" \
      --output "$out_dir" \
      --max-radius 10 \
      --connectivity 6 \
      --threshold-method {cfg["threshold"]} \
      --tau-ratio 0.05 \
      --tau-gain-rel 0.003 \
      --contacts-min 4 \
      --contacts-max 10 \
      $clahe_flag